In [11]:
# from engjax_fuctions import *
import jax.numpy as jnp
from jax.scipy.linalg import eigh
from jax import grad, jacobian
import scipy.constants as sc
import numpy as np
import MDAnalysis as mda
import re
# from jax.scipy.linalg import eigh
import jax



In [12]:
# def extract_lj_parameters(path):
#     """
#     Extracts c6 and c12 Lennard-Jones parameters from the topology file.

#     Parameters:
#     ----------
#     path : str
#         The path to the topology file in text format.

#     Returns:
#     -------
#     lj_params : dict
#         A dictionary with atom types as keys and their LJ parameters (c6 and c12) as values.
#     """
#     with open(path, 'r') as file:
#         data = file.read()

#     # Regular expression pattern for LJ_SR types
#     lj_sr_pattern = r"type=(\S+).*?c6=\s*([0-9\.\-eE+]+).*?c12=\s*([0-9\.\-eE+]+)"

#     # Find matches for LJ_SR
#     lj_sr_matches = re.findall(lj_sr_pattern, data)

#     # Process matches into a dictionary
#     lj_params = {}
#     for match in lj_sr_matches:
#         atom_type, c6, c12 = match
#         lj_params[atom_type] = {
#             "c6": float(c6),
#             "c12": float(c12)
#         }

#     return lj_params

In [13]:
import jax.numpy as jnp
from jax.scipy.linalg import eigh
from jax import grad
import re
import MDAnalysis as mda

# Extract LJ parameters from topology file
def extract_lj_parameters(path):
    with open(path, 'r') as file:
        data = file.read()
    lj_sr_pattern = r"type=(\S+).*?c6=\s*([0-9\.\-eE+]+).*?c12=\s*([0-9\.\-eE+]+)"
    lj_sr_matches = re.findall(lj_sr_pattern, data)
    lj_params = {atom_type: {"c6": float(c6), "c12": float(c12)} for atom_type, c6, c12 in lj_sr_matches}
    return lj_params

In [14]:
tpr_file = "/Users/sss/Documents/EnergyGap_project/ENGgromax_output_files/md.tpr"
xtc_file = "/Users/sss/Documents/EnergyGap_project/ENGgromax_output_files/md.xtc"
lj_contents = "/Users/sss/Documents/EnergyGap_project/ENGgromax_output_files/tpr_contents.txt"
lj_params = extract_lj_parameters(lj_contents)

In [15]:
u = mda.Universe(tpr_file, xtc_file )
print(f"{u}, unlike in the paper it reports 39768 atoms\n  However the protein atoms is {len(u.select_atoms('protein'))} same as the paper" )
time_step = u.trajectory.dt 
print(f"The time step for the trajectories is : {time_step} ps" )
print(f"The system has {u.trajectory.n_frames} frames")
print(f"The total simulation time is 81 x 5 = {len(u.trajectory)*u.trajectory.dt} ps\n   The paper reports saving a frame every 400ps which mean in this work we can only reproduce the first frame , equivilant to frame number 200.\n   However to test the code, more frames are needed, therefore, a frame every 40ps will be selected. ")
protein = u.select_atoms("protein")

protein_residues = protein.residues
print(f"The system has {len(protein_residues)} residues")
n_residues = len(protein_residues)
for residue in protein_residues:
    print(f"{ residue.resid}, {residue.resname}")

<Universe with 8339 atoms>, unlike in the paper it reports 39768 atoms
  However the protein atoms is 304 same as the paper
The time step for the trajectories is : 5.0 ps
The system has 81 frames
The total simulation time is 81 x 5 = 405.0 ps
   The paper reports saving a frame every 400ps which mean in this work we can only reproduce the first frame , equivilant to frame number 200.
   However to test the code, more frames are needed, therefore, a frame every 40ps will be selected. 
The system has 20 residues
1, ASN
2, LEU
3, TYR
4, ILE
5, GLN
6, TRP
7, LEU
8, LYS
9, ASP
10, GLY
11, GLY
12, PRO
13, SER
14, SER
15, GLY
16, ARG
17, PRO
18, PRO
19, PRO
20, SER


In [16]:
# # def compute_distance_matrix(u, residues):
# #     """
# #     Compute a matrix of distances between the centers of mass of the residues
# #     using JAX-compatible operations.

# #     Parameters
# #     ----------
# #     u : MDAnalysis.Universe
# #         The MDAnalysis universe containing the trajectory.
# #     residues : list of MDAnalysis.core.groups.Residue
# #         A list of residues in the universe.

# #     Returns
# #     -------
# #     distance_matrix : jjnp.ndarray
# #         A symmetric matrix of distances between the centers of mass of the residues.
# #     residue_identifiers : list
# #         List of identifiers corresponding to the residues in the distance matrix.
# #     """
# #     # Calculate the centers of mass of all residues
# #     coms = jnp.array([residue.atoms.center_of_mass() for residue in residues])
# #     # Log the residue identifiers
# #     residue_identifiers = [(residue.resid, residue.resname) for residue in residues]
# #     # Compute the distance matrix
# #     distance_matrix = jax.vmap(
# #         lambda x: jax.vmap(lambda y: jnp.linalg.norm(x - y))(coms)
# #     )(coms)
# #     return distance_matrix, residue_identifiers



# def compute_distance_matrix(coords, residues):
#     """
#     Compute a matrix of distances between atoms from given coordinates.
#     """
#     # Extract residue centers of mass
#     start_idx = 0
#     coms = []
#     for residue in residues:
#         n_atoms = len(residue.atoms)
#         com = jnp.mean(coords[start_idx: start_idx + n_atoms], axis=0)
#         coms.append(com)
#         start_idx += n_atoms

#     coms = jnp.array(coms)

#     # Compute distance matrix
#     distance_matrix = jax.vmap(
#         lambda x: jax.vmap(lambda y: jnp.linalg.norm(x - y))(coms)
#     )(coms)
    
#     return distance_matrix, residues


 





# def compute_interaction_energy_matrix(distance_matrix, residues, lj_params):   #, dielectric=78.5
#     """
#     Compute interaction energy matrix with Lennard-Jones and Coulombic interactions,
#     adjusted for water as the dielectric medium and units in kJ/mol.

#     Parameters
#     ----------
#     distance_matrix : jnp.ndarray
#         Matrix of distances between residues.
#     residues : list
#         List of residue objects.
#     lj_params : dict
#         Dictionary containing LJ parameters extracted from the topology file.
#     dielectric : float
#         Dielectric constant of the medium, default is 78.5 for water.

#     Returns
#     -------
#     energy_matrix : jnp.ndarray
#         Symmetric matrix of interaction energies.
#     """
#     n_residues = len(residues)
#     energy_matrix = jnp.zeros((n_residues, n_residues))
#     # conversion_factor = 1 / (4 * jnp.pi * sc.epsilon_0 * dielectric) * sc.elementary_charge**2 / sc.Avogadro / 1000  # kJ/mol
#     conversion_factor = 138.935485  # 1 / (4 * jnp.pi * sc.epsilon_0 * dielectric) # kJ/mol
#     for i in range(n_residues):
#         for j in range(i + 1, n_residues):
#             E_vdw = 0.0
#             E_coul = 0.0
#             for atom_i in residues[i].atoms:
#                 for atom_j in residues[j].atoms:
#                     r = jnp.linalg.norm(atom_i.position - atom_j.position)
#                     if r > 0:
#                         # Lennard-Jones parameters
#                         type_i = atom_i.type
#                         type_j = atom_j.type
#                         if type_i in lj_params and type_j in lj_params:
#                             c6_i = lj_params[type_i]["c6"]
#                             c12_i = lj_params[type_i]["c12"]
#                             c6_j = lj_params[type_j]["c6"]
#                             c12_j = lj_params[type_j]["c12"]

#                             # Compute sigma and epsilon
#                             sigma_i = (c12_i / c6_i) ** (1 / 6)
#                             epsilon_i = c6_i ** 2 / (4 * c12_i)
#                             sigma_j = (c12_j / c6_j) ** (1 / 6)
#                             epsilon_j = c6_j ** 2 / (4 * c12_j)

#                             # Apply Lorentz-Berthelot mixing rules
#                             sigma_ij = (sigma_i + sigma_j) / 2
#                             epsilon_ij = jnp.sqrt(epsilon_i * epsilon_j)

#                             # Lennard-Jones potential
#                             E_vdw += 4 * epsilon_ij * ((sigma_ij / r)**12 - (sigma_ij / r)**6)

#                         # Coulomb interaction
#                         q_i = atom_i.charge
#                         q_j = atom_j.charge
#                         E_coul += conversion_factor * (q_i * q_j) / r

#             # Total interaction energy
#             # energy_matrix[i, j] = E_vdw + E_coul
#             # energy_matrix[j, i] = energy_matrix[i, j]

#             total_energy = E_vdw + E_coul
#             energy_matrix = energy_matrix.at[i, j].set(total_energy)
#             energy_matrix = energy_matrix.at[j, i].set(total_energy)  # Ensure symmetry

#     return energy_matrix




# # def coarse_tuning_loop_combined(eng_time_series, sdeng_time_series, percentages):
# #     """
# #     Coarse-grain tuning loop to compute folded probability based on combined ENG and SDENG thresholds.

# #     Parameters:
# #     -----------
# #     eng_time_series : jjnp.ndarray
# #         Array of ENG(t) values across all frames.
# #     sdeng_time_series : jjnp.ndarray
# #         Array of SDENG(t) values across all frames.
# #     percentages : list
# #         List of percentages for threshold adjustment.

# #     Returns:
# #     --------
# #     folded_probabilities : list
# #         Folded probabilities for each percentage.
# #     folded_counts : list
# #         Number of frames satisfying the combined thresholds for each percentage.
# #     """
# #     # Compute statistical measures
# #     eng_max = jjnp.max(eng_time_series)
# #     sdeng_min = jjnp.min(sdeng_time_series)
# #     msd = jjnp.std(eng_time_series)  # Mean standard deviation

# #     # Initialize results
# #     folded_counts = []
# #     folded_probabilities = []

# #     # Loop over each percentage
# #     for n in percentages:
# #         # Calculate thresholds
# #         eng_threshold = eng_max - (n / 100) * msd
# #         sdeng_threshold = sdeng_min + (n / 100) * msd

# #         # Count frames satisfying both thresholds
# #         valid_frames = jjnp.logical_and(eng_time_series > eng_threshold, sdeng_time_series < sdeng_threshold)
# #         folded_count = jjnp.sum(valid_frames)

# #         # Calculate folded probability
# #         total_frames = u.trajectory.n_frames#len(eng_time_series)
# #         folded_probability = folded_count / total_frames

# #         # Append results
# #         folded_counts.append(int(folded_count))
# #         folded_probabilities.append(float(folded_probability))

# #     return folded_probabilities, folded_counts




# def modulate_weights_with_probability(eng_time_series, sdeng_time_series, frame_index, alpha, beta, percentages):
#     """
#     Modulates weights based on probabilities derived from ENG and SDENG thresholds.

#     Parameters:
#     ----------
#     eng_time_series : jjnp.ndarray
#         Array of ENG(t) values across all frames.
#     sdeng_time_series : jjnp.ndarray
#         Array of SDENG(t) values across all frames.
#     frame_index : int
#         Current frame index.
#     alpha : float
#         Current weight for ENG(T).
#     beta : float
#         Current weight for SDENG(T).
#     percentages : list
#         List of percentages for threshold adjustment.

#     Returns:
#     -------
#     new_alpha : float
#         Updated weight for ENG(T).
#     new_beta : float
#         Updated weight for SDENG(T).
#     """

#     # Compute statistical measures
#     eng_max = jnp.max(eng_time_series)
#     sdeng_min = jnp.min(sdeng_time_series)
#     msd = jnp.std(eng_time_series)  # Mean standard deviation

#     # Dynamic thresholds for the current frame
#     n = percentages[frame_index % len(percentages)]  #or percentages = list(range(0, 101, 1)) # Cyclic index for thresholds
#     eng_threshold = eng_max - (n / 100) * msd
#     sdeng_threshold = sdeng_min + (n / 100) * msd

#     # Calculate probabilities for the current frame
#     eng_prob = eng_time_series[frame_index] > eng_threshold
#     sdeng_prob = sdeng_time_series[frame_index] < sdeng_threshold

#     # Modulate weights
#     if eng_prob:
#         alpha *= 1.1  # Increase alpha if ENG(T) exceeds the threshold
#     if sdeng_prob:
#         beta *= 0.9  # Decrease beta if SDENG(T) is below the threshold

#     return alpha, beta




# # Function to compute the eigenvalues and eigenvectors of the energy matrix
# def compute_eigen(energy_matrix_jax):
#     eigenvalues, eigenvectors = eigh(energy_matrix_jax)
#     return eigenvalues, eigenvectors



# def compute_eng_sdeng(eigenvalues):
#     """
#     Compute ENG(T) and SDENG(T) from eigenvalues.

#     Parameters:
#     ----------
#     eigenvalues : jjnp.ndarray
#         Eigenvalues of the energy matrix.

#     Returns:
#     -------
#     eng_t : float
#         ENG(T) value.
#     sdeng_t : float
#         SDENG(T) value.
#     """
#     spectral_gap = eigenvalues[1] - eigenvalues[0]  # λ2 - λ1
#     avg_separation = jnp.mean(jnp.diff(eigenvalues))  # Average difference between eigenvalues
#     eng_t = spectral_gap / avg_separation if avg_separation > 0 else 0.0  # Avoid division by zero
#     sdeng_t = jnp.std(eigenvalues)
#     return eng_t, sdeng_t




# def compute_cv(eng_t, sdeng_t, alpha=1.0, beta=1.0):
#     """
#     Compute the CV as a weighted combination of ENG(T) and SDENG(T).

#     Parameters:
#     ----------
#     eng_t : float
#         ENG(T) value for the current frame.
#     sdeng_t : float
#         SDENG(T) value for the current frame.
#     alpha : float
#         Weighting factor for ENG(T).
#     beta : float
#         Weighting factor for SDENG(T).

#     Returns:
#     -------
#     cv : float
#         The value of the collective variable for the current frame.
#     """
#     return alpha * eng_t - beta * sdeng_t




# # Corrected Gradient Function with Explicit Dependency
# def compute_cv_gradient(coords, protein_residues, lj_params, alpha, beta, u):
#     """
#     Corrected function to compute CV gradient with explicit coordinate dependency.
#     """

#     def cv_function(coords):
#         # Recompute energy matrix from coordinates
#         distance_matrix, _ = compute_distance_matrix(u, protein_residues)
#         energy_matrix = compute_interaction_energy_matrix(distance_matrix, protein_residues, lj_params)
#         energy_matrix_jax = jnp.array(energy_matrix)

#         # Recompute eigenvalues
#         eigenvalues, _ = compute_eigen(energy_matrix_jax)

#         # Recompute ENG(T) and SDENG(T)
#         eng_t, sdeng_t = compute_eng_sdeng(eigenvalues)

#         # Compute CV based on ENG and SDENG
#         return compute_cv(eng_t, sdeng_t, alpha, beta)

#     # Differentiate with respect to `coords`
#     gradient = grad(cv_function)(coords)
#     return gradient




# # def compute_cv_gradient(coords, energy_matrix, eng_t, sdeng_t, alpha, beta, preprocess_residues, lj_params):
# #     def cv_function(coords):
# #         # Energy matrix depends on coordinates
# #         distance_matrix, _ = compute_distance_matrix(coords, protein_residues)
# #         energy_matrix = compute_interaction_energy_matrix(distance_matrix, protein_residues, lj_params)

# #         # ENG and SDENG depend on the energy matrix
# #         eigenvalues, _ = compute_eigen(jjnp.array(energy_matrix))
# #         eng_t, sdeng_t = compute_eng_sdeng(eigenvalues)

# #         # CV depends on ENG, SDENG, alpha, beta
# #         return compute_cv(eng_t, sdeng_t, alpha, beta)

# #     # Compute gradient
# #     gradient = grad(cv_function)(coords)
# #     return gradient







In [17]:


# Compute the distance matrix using residue centers of mass
def compute_distance_matrix(coords, residues):
    start_idx = 0
    coms = [jnp.mean(coords[start_idx: start_idx + len(residue.atoms)], axis=0)
            for residue in residues]
    coms = jnp.array(coms)
    distance_matrix = jax.vmap(lambda x: jax.vmap(lambda y: jnp.linalg.norm(x - y))(coms))(coms)
    return distance_matrix

# Compute interaction energy matrix
def compute_interaction_energy_matrix(distance_matrix, residues, lj_params):
    n_residues = len(residues)
    energy_matrix = jnp.zeros((n_residues, n_residues))
    conversion_factor = 138.935485

    for i in range(n_residues):
        for j in range(i + 1, n_residues):
            E_vdw = 0.0
            E_coul = 0.0
            for atom_i in residues[i].atoms:
                for atom_j in residues[j].atoms:
                    r = jnp.linalg.norm(atom_i.position - atom_j.position)
                    if r > 0:
                        type_i = atom_i.type
                        type_j = atom_j.type
                        if type_i in lj_params and type_j in lj_params:
                            c6_i = lj_params[type_i]["c6"]
                            c12_i = lj_params[type_i]["c12"]
                            c6_j = lj_params[type_j]["c6"]
                            c12_j = lj_params[type_j]["c12"]

                            sigma_i = (c12_i / c6_i)**(1/6)
                            epsilon_i = c6_i**2 / (4 * c12_i)
                            sigma_j = (c12_j / c6_j)**(1/6)
                            epsilon_j = c6_j**2 / (4 * c12_j)

                            sigma_ij = (sigma_i + sigma_j) / 2
                            epsilon_ij = jnp.sqrt(epsilon_i * epsilon_j)

                            E_vdw += 4 * epsilon_ij * ((sigma_ij / r)**12 - (sigma_ij / r)**6)

                        E_coul += conversion_factor * (atom_i.charge * atom_j.charge) / r

            total_energy = E_vdw + E_coul
            energy_matrix = energy_matrix.at[i, j].set(total_energy)
            energy_matrix = energy_matrix.at[j, i].set(total_energy)

    return energy_matrix

# Compute eigenvalues and eigenvectors
def compute_eigen(energy_matrix):
    eigenvalues, eigenvectors = eigh(energy_matrix)
    return eigenvalues, eigenvectors

# Compute ENG and SDENG from eigenvalues
def compute_eng_sdeng(eigenvalues):
    spectral_gap = eigenvalues[1] - eigenvalues[0]
    avg_separation = jnp.mean(jnp.diff(eigenvalues))
    eng_t = spectral_gap / avg_separation if avg_separation > 0 else 0.0
    sdeng_t = jnp.std(eigenvalues)
    return eng_t, sdeng_t

# Compute the CV
def compute_cv(eng_t, sdeng_t, alpha, beta):
    return alpha * eng_t - beta * sdeng_t

# Modulate weights based on probabilities
def modulate_weights_with_probability(eng_time_series, sdeng_time_series, frame_index, alpha, beta, percentages):
    eng_max = jnp.max(eng_time_series)
    sdeng_min = jnp.min(sdeng_time_series)
    msd = jnp.std(eng_time_series)

    n = percentages[frame_index % len(percentages)]
    eng_threshold = eng_max - (n / 100) * msd
    sdeng_threshold = sdeng_min + (n / 100) * msd

    eng_prob = eng_time_series[frame_index] > eng_threshold
    sdeng_prob = sdeng_time_series[frame_index] < sdeng_threshold

    if eng_prob:
        alpha *= 1.1
    if sdeng_prob:
        beta *= 0.9

    return alpha, beta

# Compute CV gradient with respect to coordinates
def compute_cv_gradient(coords, protein_residues, lj_params, alpha, beta):
    def cv_function(coords):
        distance_matrix = compute_distance_matrix(coords, protein_residues)
        energy_matrix = compute_interaction_energy_matrix(distance_matrix, protein_residues, lj_params)
        eigenvalues, _ = compute_eigen(energy_matrix)
        eng_t, sdeng_t = compute_eng_sdeng(eigenvalues)
        return compute_cv(eng_t, sdeng_t, alpha, beta)

    gradient = grad(cv_function)(coords)
    return gradient

# # Main Simulation Setup
# tpr_file = "path_to_tpr_file"
# xtc_file = "path_to_xtc_file"
# lj_contents = "path_to_topology_file"

# lj_params = extract_lj_parameters(lj_contents)
# u = mda.Universe(tpr_file, xtc_file)
# protein = u.select_atoms("protein")
# protein_residues = protein.residues

time_step = u.trajectory.dt
selected_frames = range(0, u.trajectory.n_frames, int(40 / time_step))
percentages = list(range(0, 101, 1))

alpha, beta = 1.0, 1.0
eng_time_series = jnp.zeros(len(selected_frames))
sdeng_time_series = jnp.zeros(len(selected_frames))

for frame_idx, frame in enumerate(selected_frames):
    u.trajectory[frame]
    coords = jnp.array([atom.position for atom in protein.atoms])
    distance_matrix = compute_distance_matrix(coords, protein_residues)
    energy_matrix = compute_interaction_energy_matrix(distance_matrix, protein_residues, lj_params)
    eigenvalues, _ = compute_eigen(energy_matrix)
    eng_t, sdeng_t = compute_eng_sdeng(eigenvalues)

    eng_time_series = eng_time_series.at[frame_idx].set(eng_t)
    sdeng_time_series = sdeng_time_series.at[frame_idx].set(sdeng_t)
    cv_t = compute_cv(eng_t, sdeng_t, alpha, beta)

    gradient = compute_cv_gradient(coords, protein_residues, lj_params, alpha, beta)

    print(f"Frame {frame_idx}: ENG(T): {eng_t}, SDENG(T): {sdeng_t}, CV(T): {cv_t}, Gradient: {gradient}, Gradient Norm: {jnp.linalg.norm(gradient)}")

    alpha, beta = modulate_weights_with_probability(
        eng_time_series, sdeng_time_series, frame_idx, alpha, beta, percentages
    )


Frame 0: ENG(T): 2.725583076477051, SDENG(T): 17.3066463470459, CV(T): -14.581063270568848, Gradient Norm: 0.0
Frame 1: ENG(T): 2.580644130706787, SDENG(T): 17.492294311523438, CV(T): -14.911649703979492, Gradient Norm: 0.0
Frame 2: ENG(T): 1.3902591466903687, SDENG(T): 17.407638549804688, CV(T): -16.017379760742188, Gradient Norm: 0.0


KeyboardInterrupt: 

## Compute eigenvalues and eigenvectors

In [312]:
interval_ps = 40  # unit: ps
interval_frames = int(interval_ps / time_step)
selected_frames = list(range(0, u.trajectory.n_frames, interval_frames))
percentages = list(range(0, 101, 1))
alpha, beta = 1.0, 1.0

num_frames = len(selected_frames)
num_residues = len(protein_residues)

# Initialize arrays to store time series and results
eng_time_series = jnp.zeros(num_frames)
sdeng_time_series = jnp.zeros(num_frames)

# Main loop
for frame_index, frame in enumerate(selected_frames):
    u.trajectory[frame]  

    # Extract coordinates from the current frame
    coords = jnp.array([atom.position for atom in protein.atoms])

    # Compute gradient directly (everything happens inside this function)
    gradient = compute_cv_gradient(coords, protein_residues, lj_params, alpha, beta,u)

    # Retrieve ENG(T) and SDENG(T) from inside the gradient function
    eng_t, sdeng_t = compute_eng_sdeng(compute_eigen(compute_interaction_energy_matrix(
        compute_distance_matrix(u, protein_residues)[0], protein_residues, lj_params
    ))[0])

    # Update time series
    eng_time_series = eng_time_series.at[frame_index].set(eng_t)
    sdeng_time_series = sdeng_time_series.at[frame_index].set(sdeng_t)

    # Compute CV
    cv_t = compute_cv(eng_t, sdeng_t, alpha, beta)

    # Debugging outputs
    print(f"Frame {frame_index}:\n  ENG(T): {eng_t}, SDENG(T): {sdeng_t}, CV(T): {cv_t}")
    print(f"  Gradient & Gradient Shape: {gradient}, {gradient.shape}")

    # Modulate weights dynamically
    alpha, beta = modulate_weights_with_probability(
        eng_time_series, sdeng_time_series, frame_index, alpha, beta, percentages
    )
    print(f"CV: {cv_t}, Alpha: {alpha}, Beta: {beta}")

# Final debugging outputs
print("Final ENG Time Series:", eng_time_series)
print("Final SDENG Time Series:", sdeng_time_series)


TypeError: 'Universe' object is not subscriptable

# A repeat of the code above

### Validation and Debugging

### calculating the ENG and SDENG

In [5]:
interval_ps = 40  # unit: ps
interval_frames = int(interval_ps / time_step)
selected_frames = list(range(0, u.trajectory.n_frames, interval_frames))
percentages = list(range(0, 101, 1))
alpha, beta = 1.0, 1.0
time_step = u.trajectory.dt
num_frames = len(selected_frames)
num_residues = len(protein_residues)

# Initialize arrays to store time series and results
eng_time_series = jnp.zeros(num_frames)
sdeng_time_series = jnp.zeros(num_frames)




for frame_index, frame in enumerate(selected_frames):
    u.trajectory[frame]  
    # Extract coordinates from the current frame
    coords = jnp.array([atom.position for atom in protein.atoms])
    # Compute gradient directly (everything happens inside this function)
    gradient = compute_cv_gradient(coords, protein_residues, lj_params, alpha, beta,u)
    eng_t, sdeng_t = compute_eng_sdeng(compute_eigen(compute_interaction_energy_matrix(
        compute_distance_matrix(u, protein_residues)[0], protein_residues, lj_params
    ))[0])
    print(f"Frame {frame_index}:\n  ENG(T): {eng_t}, SDENG(T): {sdeng_t}, CV(T): {compute_cv(eng_t, sdeng_t, alpha, beta)}")
    print(f"  Gradient & Gradient Shape: {gradient}, {gradient.shape}")
    # Modulate weights dynamically
    alpha, beta = modulate_weights_with_probability(
        eng_time_series, sdeng_time_series, frame_index, alpha, beta, percentages
    )
    print(f"CV: {compute_cv(eng_t, sdeng_t, alpha, beta)}, Alpha: {alpha}, Beta: {beta}")


NameError: name 'time_step' is not defined

Immediate Next Steps:  

Folding Probability & Coarse Tuning Loop:
We need to compute the folding probability 
P
Folded
P 
Folded
​	
  for each frame. This involves comparing the computed ENG(t) and SDENG(t) values to dynamic thresholds based on percentages.
Fine Tuning Loop:
Use the correlation distance (CD) to refine the parameters 
α
α and 
β
β, controlling ENG(t) and SDENG(t). This requires:
Residue time series contributions from the eigenvector of the smallest eigenvalue.
Computing correlation coefficients and distances between the time series of ENG(t) and residue contributions.  


End Goal:
Implement ENG(t) as a Collective Variable (CV) in PLUMED:
Calculate gradients of the ENG(t) CV w.r.t. atomic coordinates 
r
r using JAX auto-differentiation.
Write the C++ implementation of ENG(t) for PLUMED using:
∂
ENG
(
t
)
∂
λ
∂λ
∂ENG(t)
​	
 , where 
λ
λ are the eigenvalues.
∂
λ
∂
M
∂M
∂λ
​	
 , where 
M
M is the interaction energy matrix.
∂
M
∂
r
∂r
∂M
​	
 , where 
r
r are the atomic coordinates.
Iterative Feedback Loops:
Fine-tune the CV parameters dynamically during the molecular dynamics simulation to capture folding and unfolding mechanisms.


### Coarse tuning Loop  
Inputs  

1. ENG(t) time series  
2. SDENG(t) time series  
3. Percentages 

Steps  

1. Compute thresholds  
2. Count frames satisfying the threshold, above eng and below sdeng  
3. calculate Pfold for each percentage  

Outputs  

1. Folded Probabilities for each percentage.  
2. Counts of frames satisfying the thresholds


Code is here but implementation is up within the loop

1. Extract Residue Contributions  
Iterate over the frames, sum the eigenvector contributions per residue, and build the residue time series.
2. Calculate Correlation Coefficients  
Compute the correlation of each residue's time series with ENG(t).
3. Calculate Correlation Distance (CD)  
Use the formula above to compute CD for each residue.
4. Debug and Output the Results  
Verify that CD values make sense and are in the expected range [0, 2].